In [1]:
#智能体们自己决定谁讲话。我们可以通过让每个智能体竞标发言来实现这一点。
#竞标最高的智能体将发言。
from langchain import PromptTemplate
import re
import tenacity
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import RegexParser
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

In [2]:
import sys
sys.path.append('/home/kit/kit/donson_intern/langchain/Dungeons&dragons')
from DialogueAgent import DialogueAgent
from DialogueSimulator import DialogueSimulator

sys.path.append('/home/kit/kit/donson_intern/langchain')
from ChatGLM3 import ChatGLM3
model_path = "/data/tds/ChatGLM3/chatglm3-6b"
llm = ChatGLM3()
llm.load_model(model_name_or_path=model_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template

    def bid(self) -> str:
#让智能体提交发言出价
        prompt = PromptTemplate(
            input_variables=['message_history', 'recent_message'],
            template = self.bidding_template
        ).format(
            message_history=''.join(self.message_history),
            recent_message=self.message_history[-1])
        bid_string = self.model(prompt)
        return bid_string

In [4]:
character_names = ["Donald Trump", "Kanye West", "Elizabeth Warren"]
topic = "transcontinental high speed rail"
word_limit = 50

In [5]:
game_description = f"""Here is the topic for the presidential debate: {topic}.
The presidential candidates are: {', '.join(character_names)}."""
player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each presidential candidate.")

def generate_character_description(character_name):
    character_specifier_prompt_human_message = HumanMessage(content=
            f"""{game_description}
Please reply with a creative description of the presidential candidate, {character_name}, in {word_limit} words or less, that emphasizes their personalities. 
Speak directly to {character_name}.
Do not add anything else."""
            )
    prompt = player_descriptor_system_message.content + '/n' +character_specifier_prompt_human_message.content
    character_description = llm(prompt)
    return character_description
 
def generate_character_header(character_name, character_description):
    return f"""{game_description}
Your name is {character_name}.
You are a presidential candidate.
Your description is as follows: {character_description}
You are debating the topic: {topic}.
Your goal is to be as creative as possible and make the voters think you are the best candidate.
"""
 
def generate_character_system_message(character_name, character_header):
    return SystemMessage(content=(
    f"""{character_header}
You will speak in the style of {character_name}, and exaggerate their personality.
You will come up with creative ideas related to {topic}.
Do not say the same things over and over again.
Speak in the first person from the perspective of {character_name}
For describing your own body movements, wrap your description in '\*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
 """
    ))

character_descriptions = [generate_character_description(character_name) for character_name in character_names]
character_headers = [generate_character_header(character_name, character_description) for character_name, character_description in zip(character_names, character_descriptions)]
character_system_messages = [generate_character_system_message(character_name, character_headers) for character_name, character_headers in zip(character_names, character_headers)]                                                                                                                       

In [6]:
for character_name, character_description, character_header, character_system_message in zip(character_names, character_descriptions, character_headers, character_system_messages):
    print(f'{character_name} Description:')
    print(f'{character_description}')
    print(f'{character_header}')
    print(f'{character_system_message.content}')


Donald Trump Description:
"Donald Trump is a billionaire businessman and former reality TV star who is known for his outspoken and polarizing personality. He is a strong leader who stands up for his beliefs, even if they go against the norm. He is not afraid to speak his mind and always has a unique perspective on the world."
Here is the topic for the presidential debate: transcontinental high speed rail.
The presidential candidates are: Donald Trump, Kanye West, Elizabeth Warren.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: "Donald Trump is a billionaire businessman and former reality TV star who is known for his outspoken and polarizing personality. He is a strong leader who stands up for his beliefs, even if they go against the norm. He is not afraid to speak his mind and always has a unique perspective on the world."
You are debating the topic: transcontinental high speed rail.
Your goal is to be as creative as possible and make the v

In [7]:
# 出价解析器
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return 'Your response should be an integer delimited by angled brackets, like this: <int>.'     

bid_parser = BidOutputParser(
    regex=r'<(\d+)>', 
    output_keys=['bid'],
    default_output_key='bid')

In [8]:
def generate_character_bidding_template(character_header):
    bidding_template = (
    f"""{character_header}
/```
{{message_history}}
/```
On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.
/```
{{recent_message}}
/```
{bid_parser.get_format_instructions()}
Do nothing else.
 """)
    return bidding_template

character_bidding_templates = [generate_character_bidding_template(character_header) for character_header in character_headers]                                   

for character_name, bidding_template in zip(character_names, character_bidding_templates):
    print(f'{character_name} Bidding Template:')
    print(bidding_template)

Donald Trump Bidding Template:
Here is the topic for the presidential debate: transcontinental high speed rail.
The presidential candidates are: Donald Trump, Kanye West, Elizabeth Warren.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: "Donald Trump is a billionaire businessman and former reality TV star who is known for his outspoken and polarizing personality. He is a strong leader who stands up for his beliefs, even if they go against the norm. He is not afraid to speak his mind and always has a unique perspective on the world."
You are debating the topic: transcontinental high speed rail.
Your goal is to be as creative as possible and make the voters think you are the best candidate.

/```
{message_history}
/```
On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.
/```
{recent_message}
/```
Your response should be an integer delimited by a

In [9]:
topic_specifier_prompt_system_message = SystemMessage(content="You can make a task more specific.")
topic_specifier_prompt_human_message = HumanMessage(content=
        f"""{game_description}
You are the debate moderator.
Please make the debate topic more specific. 
Frame the debate topic as a problem to be solved.
Be creative and imaginative.
Please reply with the specified topic in {word_limit} words or less. 
Speak directly to the presidential candidates: {*character_names,}.
Do not add anything else."""
        )
prompt = topic_specifier_prompt_system_message.content + '/n' + topic_specifier_prompt_human_message.content
specified_topic = llm(prompt)

print(f"Original topic:{topic}")
print(f"Detailed topic:{specified_topic}")

Original topic:transcontinental high speed rail
Detailed topic:"Greetings, candidates. The problem we face today is the lack of efficient transportation infrastructure across our country. How do you propose to solve this issue and bring high-speed rail to the table?"


In [10]:
@tenacity.retry(stop=tenacity.stop_after_attempt(2),
                    wait=tenacity.wait_none(),  # No waiting time between retries
                    retry=tenacity.retry_if_exception_type(ValueError),
                    before_sleep=lambda retry_state: print(f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."),
                    retry_error_callback=lambda retry_state: 0) # Default value when all retries are exhausted
 
# 要求智能体出价，并且解析答案为正确格式
def ask_for_bid(agent) -> str:
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)['bid'])
    return bid

In [11]:
import numpy as np
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)
 
    # 相同出价，随机选取
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)
    print('Bids:')
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f'- {agent.name} bid: {bid}')
        if i == idx:
            selected_name = agent.name
    print(f'Selected: {selected_name}')
    print('')
    return idx

In [12]:
characters = []
for character_name, character_system_message, bidding_template in zip(character_names, character_system_messages, character_bidding_templates):
    characters.append(BiddingDialogueAgent(
        name=character_name,
        system_message=character_system_message,
        model=llm,
        bidding_template=bidding_template,
    ))

In [13]:
max_iters = 10
n = 0
simulator = DialogueSimulator( 
    agents=characters, 
    selection_function=select_next_speaker 
)
simulator.reset()
simulator.inject('Debate Moderator', specified_topic)
print(f"(Debate Moderator): {specified_topic}")
print('')
while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('')
    n += 1

(Debate Moderator): "Greetings, candidates. The problem we face today is the lack of efficient transportation infrastructure across our country. How do you propose to solve this issue and bring high-speed rail to the table?"

ValueError occurred: invalid literal for int() with base 10: '[5]', retrying...
ValueError occurred: invalid literal for int() with base 10: '[5]', retrying...
ValueError occurred: invalid literal for int() with base 10: '[5]', retrying...
Bids:
- Donald Trump bid: 0
- Kanye West bid: 0
- Elizabeth Warren bid: 0
Selected: Donald Trump

(Donald Trump): *I believe we need to build a transcontinental high speed rail system that will connect our great cities from coast to coast. It's time to get rid of the slow and expensive air travel and have a faster, more convenient way to travel. We need to invest in our infrastructure and make it the best it can be. Believe me, it will be tremendous.

Bids:
- Donald Trump bid: 10
- Kanye West bid: 1
- Elizabeth Warren bid: 1
Sel